# Дерево Фенвика

- Структура, позволяющая на многих задачах заменить собой дерево отрезков
- Работает намного быстрее (на сумме в ~5 раз быстрее ДО с e-maxx.ru)
- Пишется быстрее и проще — трудно запомнить только две формулы
- Очень легко обобщается на б*о*льшие размерности, дружит с хэш-таблицами и другими внутренними структурами
- Занимает памяти ровно столько же, сколько бы потребовалось на хранение такого же массива

 Пусть дан массив $a$ длины $n$. Деревом фенвика будем называть массив $t$ той же длины, который объявим так: 
 
 $$ t_i = \sum_{k=F(i)}^i a_k $$
 
 Для $F$ выполнено $F(i) \leq i$. Конкретно её определим потом.
 
 Когда нам нужна сумма на отрезке, мы сводим этот запрос к двум суммам на префиксе, каждый из которых будем считать по этой формуле:
 
 $$ sum(k) = t_k + sum(F(k)-1) $$
 
 Когда мы изменяем $k$-ю ячейку исходного массива, мы обновляем все $t_i$, в которых учтена эта ячейка.
 
 $F$ можно выбрать так, что и «спусков» при подсчете суммы, и интересных нам $t_i$ при обновлении будет будет $O(\log n)$.

В качестве $F$ популярны две функции:

* $F_1(x) =$ `x & (x+1)`
* $F_2(x) =$ `x - (x & -x) + 1`

Первый вариант описан на викиконспектах и емаксе и поэтому более известен.

Второй, как мы дальше увидим, более простой для запоминания и кодинга, а так же более гибкий — например, там можно делать бинпоиск по префиксным суммам. Он же и используется в статье.

Наверное, меньше четверти умеющих писать эту структуру полностью понимает, как она работает. Анализ действительно весьма сложный. Он приведен в конце статьи.

## Реализация

Из-за того, что $F(0) = 1 > 0$ и поэтому $[0, F(0)]$ не является корректным отрезком, нам будет удобнее хранить массив в 1-индексации и не использовать $t_0$.

In [0]:
int t[maxn];

// возвращает сумму на префиксе
int sum (int r) {
    int res = 0;
    for (; r > 0; r -= r & -r)
        res += t[r];
    return res;
}

int sum (int l, int r) {
    return sum(r) - sum(l-1);
}

// обновляет нужные t
void add (int k, int x) {
    for (; k <= n; k += k & -k)
        t[k] += x;
}

Отмечу красивую симметрию в формулах `r += r & -r` и `k -= k & -k`, которой нет в «традиционной» реализации.

## 2d, 3d...

> $k$-мерное дерево Фенвика пишется в $(k+1)$ строчку

Нужно добавить всего одну такую же строчку в `sum`, `add`, а также при подсчете суммы на прямоугольнике вместо двух запросов к префиксным суммам использовать четыре.

`sum` перепишется следующим образом:

In [0]:
int sum (int r1, int r2) {
    int res = 0;
    for (int i = r1; i > 0; i -= i & -i)
        for (int j = r2; j > 0; j -= j & -j)
            ans += t[i][j];
    return res;
}

Автор в своё время следующим образом решил какую-то задачу на 2d-сумму с USACO 2017: https://pastebin.com/DPemaJeW

## Бинпоиск

Оказывается, можно производить бинарный поиск (точнее, спуск) по префиксным суммам за $O(\log n)$.

In [0]:
// возвращает индекс, на котором сумма уже больше
int lower_bound (int s) {
    int k = 0;
    for (int l = logn; l >= 0; l--) {
        if (k + (1<<l) <= n && t[k + (1<<l)] < s) {
            k += (1<<l);
            s -= t[k];
        }
    }
    return k;
}

Если знать, что $F(x)$ удаляет последний бит $x$, то принцип понятен: просто делается спуск по бинарному дереву, как в ДО.

## Ограничения

Дерево Фенвика можно использовать, когда наша операция обратима, а также когда трюк с префиксными суммами работает. Это обычно простые операции типа суммы, xor, умножения по модулю (если гарантируется, что на этот модуль ничего не делится). Минимум и gcd, отложенные операции и персистентность прикрутить в общем случае уже не получится — тогда уже нужно писать ДО.

## Почему работает

(Раздел не совсем дописан.)

`F(x) = x - (x & -x) + 1`. Поймем, что означает `x & -x`.

**Лемма**. `x & -x` возвращает последний единичный бит в двоичной записи x.

Как в компах хранятся инты? Чтобы процессор не ифал + и -, их хранят просто как бы по модулю $2^k$, а первый бит отвечает за знак. Поэтому когда мы хотим узнать, как выглядит отрицательное число, нужно его вычесть из нуля: $-x = 0-x = 2^k-x$.

Рассмотрим пример.

$
\begin{align}
  \begin{aligned}
    +90 = 2+8+16+64  & = 10110_2 \\
    -90 = 00000_2 - 10110_2 & = 01010_2 \\
    +90 \text{ & } -90 & = 00010_2 \\ 
  \end{aligned}
\end{align}
$

Когда мы вычитаем, мы идем справа налево. Первые сколько-то нулей с конца ими же и останутся. Потом, на самом младшем единичном бите, мы «займём» много единиц, так что весь префикс станет единицами. Потом отменятся ровно те из них, которые были единицами в исходном числе.

Делаем &: в каком-то префиксе все биты будут противоположными, младший единичный бит останется, а на суффиксе все как было нулями, так и осталось. Выживет только этот самый младший единичный бит.

Теперь сразу понятно, почему `sum` будет работать за логарифм — каждый раз мы делаем `x -= x & -x`, то есть удаляем один бит.

Теперь сложная (для понимания) часть — как делать add.
Какие $t_i$ содержат $k$-й элемент? Для них должно выполняться `i >= k > i - (i & -i)`. 

Будем перебирать префиксы TODO

Мы знаем, что $t_i$ вложены друг в друга. Минимальный подходящий $i$ равен $k$. Какой следующий? Нам нужно для каждого $i$ уметь находить его непосредственного родителя.

Можно представить дерево так: ячейка 2^k содержит все

TODO: сначала объявить дерево как дерево и описать его структуру словами, а потом уже показывать реализацию.

## Название

Потому что $F$ использует битовые операции, по-английски структура называется «Binary Indexed Tree».

Почему дерево Фенвика — дерево? Тут нужно немного воображения. На самом деле BIT в общем случае — набор деревьев.

Можно показать, что множества элементов, учтенных в $t_i$ и $t_j$, либо не пересекаются, либо одно является подмножеством другого. Значит, между $t_i$ можно ввести отношение вложенности, и тогда дерево Фенвика можно представить как набор деревьев (не обязательно бинарных).

В частном случае, когда длина массива равна $2^k$, то дерево будет только одно.